<a href="https://colab.research.google.com/github/Lamino1788/TCC_TFT/blob/main/Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-forecasting
!pip install pytorch-lightning
!pip install yfinance


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.8/155.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_ext

In [2]:
import torch
from torchmetrics import Metric

from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, QuantileLoss, MAPE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
import yfinance as yf
import pandas as pd
import pytorch_lightning as pl
import numpy as np


from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss, RMSE


import warnings
from sklearn.exceptions import DataConversionWarning

from pytorch_lightning.tuner import Tuner
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

warnings.filterwarnings(action='ignore', category=UserWarning)

import logging
import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
max_encoder_length = 126
max_prediction_length = 10

In [5]:
class MyMAE(Metric):
  global max_prediction_length
  def __init__(self):
      super().__init__(dist_sync_on_step=False)
      self.add_state("loss", default=torch.tensor(0.0), dist_reduce_fx="sum")

  def update(self, preds, target):
      # Ensure the inputs are tensors
      # Compute cumulative product (remove `dim=1` argument)

      preds = preds.view(int(len(preds)/max_prediction_length),max_prediction_length)
      preds = preds.sum(dim=1)
      target = target.view(int(len(target)/max_prediction_length),max_prediction_length)
      target = target.sum(dim=1)

      mae = abs(preds - target).mean()

      self.loss +=  mae

  def compute(self):
      # Compute final metric
      return self.loss

In [6]:
def add_macd(df, short_period=12, long_period=26, signal_period=9): #Moving Average Convergence Divergence
    exp1 = df['Close'].ewm(span=short_period, adjust=False).mean()
    exp2 = df['Close'].ewm(span=long_period, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    df['MACD'] = macd
    df['MACD_Signal'] = signal
    return df

def add_rsi(df, period=30): #Relative Strength Index
    delta = df['Close'].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    for period in [7, 14, 22, 30, 60]:
      average_gain = up.rolling(window=period).mean()
      average_loss = abs(down.rolling(window=period).mean())
      rs = average_gain / average_loss
      rsi = 100.0 - (100.0 / (1.0 + rs))

      df[f'RSI_{period}'] = rsi
    return df

def encode_cyclical(df, col, max_val):
    df[col] = np.cos(2 * np.pi * df[col]/max_val)
    return df


In [7]:
# SPX = 'USDBRL=X' # Dollar Future
def make_SPX_hist(SPX):
  # SPX = 'ES=F'
  global unknown_reals
  SPX_data = yf.Ticker(SPX)

  # SPX_hist = SPX_data.history(start = datetime.datetime(2000,1,1), end = datetime.datetime(2023,7,18))
  SPX_hist = SPX_data.history(start = datetime.datetime(2000,1,1), end = datetime.datetime(2023,11,13))
  SPX_hist.index = pd.to_datetime(SPX_hist.index, format='%y-%m-%d').strftime('%Y-%m-%d')
  SPX_hist = SPX_hist[['Close']]

  tickers=[
      # 'ZF=F', 'ZT=F', 'ZB=F', "ZN=F",
      # "ES=F", "YM=F", "NQ=F", #"RTY=F",
      # "GC=F", "SI=F", "ZC=F", "CL=F", "SB=F", "CT=F",
      "EUR=X", "JPY=X", "GBP=X", 'BRL=X', "MXN=X", "CAD=X"
  ]
  # tickers.remove(SPX)

  for ticker in tickers:
    data = yf.Ticker(ticker)
    hist = data.history(start = datetime.datetime(2000,1,1), end = datetime.datetime(2023,11,13))
    hist.index = pd.to_datetime(hist.index, format='%y-%m-%d').strftime('%Y-%m-%d')
    SPX_hist = SPX_hist.join(pd.DataFrame(hist['Close']), rsuffix=f'_{ticker}')

  SPX_hist = np.log(SPX_hist / SPX_hist.shift(1))
  SPX_hist['Close_prediction'] = SPX_hist['Close'].shift(max_prediction_length).fillna(0)
  SPX_hist['Close_encoder'] = SPX_hist['Close'].shift(max_encoder_length).fillna(0)
  SPX_hist = SPX_hist.fillna(0).drop(list(SPX_hist.index)[0])

  SPX_hist.reset_index(inplace=True)
  SPX_hist.rename(columns={'date': 'Date'},inplace=True)

  for i in SPX_hist.index:
    SPX_hist.loc[i,'Rolling_Avg_22d'] = SPX_hist.loc[i-30:i-1, 'Close'].mean()
    SPX_hist.loc[i,'Rolling_Vol_22d'] = SPX_hist.loc[i-30:i-1, 'Close'].std()

  SPX_hist['Exponential_Avg'] = SPX_hist.Close.ewm(span=max_encoder_length).mean()
  SPX_hist = add_macd(SPX_hist)
  SPX_hist = add_rsi(SPX_hist)

  SPX_hist.fillna(0)
  SPX_hist.loc[SPX_hist.Rolling_Avg_22d == 0, 'Rolling_Avg_22d'] = SPX_hist.loc[SPX_hist.Rolling_Avg_22d == 0, 'Close']

  unknown_reals = list(SPX_hist.columns)[1:]

  SPX_hist['Ticker'] = SPX
  # SPX_hist.index = pd.to_datetime(SPX_hist.Date)
  SPX_hist.index = SPX_hist.index.astype(int)
  SPX_hist['Time_Fix'] = SPX_hist.index

  SPX_hist['Year'] = SPX_hist.Date.str.split('-').str[0].astype(int)
  SPX_hist['Month'] = SPX_hist.Date.str.split('-').str[1].astype(int)
  SPX_hist['Day'] = SPX_hist.Date.str.split('-').str[2].astype(int)

  SPX_hist = encode_cyclical(SPX_hist, 'Month', 12)
  SPX_hist = encode_cyclical(SPX_hist, 'Day', 31)

  return SPX_hist


In [8]:
SPX = "BRL"
tickers=[
      # 'ZF=F', 'ZT=F', 'ZB=F', "ZN=F",
      # "ES=F"#, "YM=F", "NQ=F", #"RTY=F",
      # "GC=F", "SI=F", "ZC=F", "CL=F", "SB=F", "CT=F",
      # "EUR=X"#, "JPY=X", "GBP=X", 'BRL=X', "MXN=X", "CAD=X"
  ]

tickers = ["BRL=X"]
SPX_hist = pd.DataFrame()
for ticker in tickers:
  print(ticker)
  ticker_hist = make_SPX_hist(ticker)
  SPX_hist = pd.concat([SPX_hist,ticker_hist],axis=0)
  # SPX_hist = SPX_hist.sort_values(by="Time_Fix")
# SPX_hist.fillna(method = "ffill", inplace=True)
# SPX = "ES=F"
# tmp2 = make_SPX_hist(SPX)
# SPX_hist = pd.concat([tmp,tmp2],axis=0)
# SPX_hist = SPX_hist.sort_values(by="Time_Fix")

BRL=X


In [9]:
SPX_hist.dropna(inplace=True)


In [10]:
high = SPX_hist.Time_Fix.max()*0.85
low = SPX_hist.Time_Fix.max()*0.7

# validation = SPX_hist.loc[(SPX_hist.Time_Fix < high) & (SPX_hist.Time_Fix > low)].reset_index(drop=True)
# test = SPX_hist.loc[(SPX_hist.Time_Fix > high)].reset_index(drop=True)
# validation = pd.concat([validation, test.iloc[:max_encoder_length]],axis=0).reset_index(drop=True)
# base = SPX_hist.loc[SPX_hist.Time_Fix < low].reset_index(drop=True)
validation = SPX_hist.loc[(SPX_hist.Year < 2020) & (SPX_hist.Year > 2016)].reset_index(drop=True)
test = SPX_hist.loc[(SPX_hist.Year > 2019)].reset_index(drop=True)
validation = pd.concat([validation, test.iloc[:max_encoder_length]],axis=0).reset_index(drop=True)
base = SPX_hist.loc[SPX_hist.Year < 2016].reset_index(drop=True)



In [11]:
training = TimeSeriesDataSet(
    base,
    time_idx="Time_Fix",
    target="Close",
    group_ids=["Ticker"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Ticker"],
    time_varying_known_reals=['Month', 'Day'],
    time_varying_unknown_reals=unknown_reals,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    min_encoder_length=max_encoder_length // 2,
    allow_missing_timesteps=True
)

val = TimeSeriesDataSet(
    validation,
    time_idx="Time_Fix",
    target="Close",
    group_ids=["Ticker"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Ticker"],
    time_varying_known_reals=['Month', 'Day'],
    time_varying_unknown_reals=unknown_reals,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    min_encoder_length=max_encoder_length // 2,
    allow_missing_timesteps=True
)

test_set = TimeSeriesDataSet(
    test,
    time_idx="Time_Fix",
    target="Close",
    group_ids=["Ticker"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Ticker"],
    time_varying_known_reals=['Month', 'Day'],
    time_varying_unknown_reals=unknown_reals,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    min_encoder_length=max_encoder_length // 2,
    allow_missing_timesteps=True

)

In [12]:
batch_size = 32
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = val.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)
test_dataloader = test_set.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)

In [13]:
import pickle
import optuna
from optuna.pruners import MedianPruner
from pytorch_lightning.callbacks import EarlyStopping
import lightning.pytorch as pl

pruner = MedianPruner(n_startup_trials=1, n_warmup_steps=5, interval_steps=1)
storage = 'sqlite:////content/drive/MyDrive/All_Round_Study.db'
study = optuna.create_study(pruner=pruner, storage=storage, direction="minimize", study_name='Currencies_Study', load_if_exists=True)
best_params = study.best_trial.params


[I 2023-11-21 21:01:05,690] Using an existing study with name 'Currencies_Study' instead of creating a new one.


In [14]:
best_tft = TemporalFusionTransformer.from_dataset(
        training,  # your training dataset
        learning_rate=best_params['learning_rate'],
        hidden_size=best_params['hidden_size'],
        attention_head_size=4,
        dropout=best_params['dropout'],
        hidden_continuous_size=best_params['hidden_continuous_size'],
        output_size=1,  # your output size
        loss=MyMAE(),
        log_interval=10,
        reduce_on_plateau_patience=4,
        lstm_layers=2,
        log_val_interval=10,
    )


In [15]:
def calculate_sharpe(test: pd.DataFrame, best_tft, max_encoder_length, max_prediction_length):
    total, bench = 1, 1
    j = 0
    signal_list, returns, pct = [], [], []

    test = test.sort_values(["Date", "Ticker"])
    output = {}
    for ticker_out in test.Ticker.unique():
      aux_df = test.loc[test.Ticker == ticker_out]
      indices = aux_df.iloc[max_encoder_length+max_prediction_length:].Time_Fix
      for i in indices:
        # try:
        rolling_df = aux_df.loc[aux_df.Time_Fix.isin(list(range(i-max_encoder_length-max_prediction_length, i)))]
        if j == max_prediction_length:
            j = 0
        if j == 0:
            test_predictions = best_tft.predict(rolling_df, mode="raw", return_x=True)
            counter = 0
            for ticker in rolling_df.Ticker.unique():
                if ticker == ticker_out:
                    pred = test_predictions.output.prediction[counter].cpu().numpy().cumsum()[-1]
                    if ticker not in output.keys():
                        output[ticker] = [pred]
                        output[ticker + "_Real"] = [rolling_df["Close_" + ticker].iloc[-max_prediction_length]]
                        output[ticker + "_Date"] = [rolling_df.loc[rolling_df.Ticker == ticker].Date.iloc[-max_prediction_length]]
                    else:
                        output[ticker] = output[ticker] + [pred]
                        output[ticker + "_Real"] = output[ticker + "_Real"] + [rolling_df["Close_" + ticker].iloc[-max_prediction_length]]
                        output[ticker + "_Date"] = output[ticker + "_Date"] + [rolling_df.loc[rolling_df.Ticker == ticker].Date.iloc[-max_prediction_length]]
                    counter = counter + 1
            j += 1
        else:
            rolling_df = test.loc[test.Time_Fix.isin(list(range(i-max_encoder_length-max_prediction_length, i)))]
            counter = 0
            for ticker in rolling_df.Ticker.unique():
                if ticker == ticker_out:
                    if ticker in output.keys():
                        output[ticker + "_Real"] = output[ticker + "_Real"] + [rolling_df["Close_" + ticker].iloc[-max_prediction_length]]
                        output[ticker + "_Date"] = output[ticker + "_Date"] + [rolling_df.loc[rolling_df.Ticker == ticker].Date.iloc[-max_prediction_length]]
                        output[ticker] = output[ticker] + [output[ticker][-1]]
                counter = counter + 1
                j += 1
        # except:
        #     continue

    return output


In [16]:
pd.options.mode.chained_assignment = None

In [17]:
name = "BRL"
version = f'{name}.pkl'
now = SPX

In [18]:
path_to_model = f"/content/drive/MyDrive/TCC/Best_Models/{SPX}/{now}/Seeds/0.pth"
best_tft.load_state_dict(torch.load(path_to_model, map_location=torch.device('cpu')))
pred = calculate_sharpe(validation, best_tft, max_encoder_length, max_prediction_length)

In [19]:
# with open(f"/content/drive/MyDrive/TCC/Best_Models/Best_Results/{SPX}/{version}", "wb") as f:
#       pickle.dump(pred, f)

In [20]:
try:
  with open(f"/content/drive/MyDrive/TCC/Best_Models/Best_Results/{SPX}/{version}", "rb") as f:
      results = pickle.load(f)
      # results = {0:[]}

except:
  results = {0:[]}
  pass

In [21]:
results.keys()

dict_keys([0])

In [22]:
for i in range(0,25):
  try:
    path_to_model = f"/content/drive/MyDrive/TCC/Best_Models/{SPX}/{now}/Seeds/{i}.pth"
    print(path_to_model)

    best_tft.load_state_dict(torch.load(path_to_model, map_location=torch.device('cpu')))
    pred = calculate_sharpe(validation, best_tft, max_encoder_length, max_prediction_length)
    results[i] = pred

    with open(f"/content/drive/MyDrive/TCC/Best_Models/Best_Results/New_Results/{SPX}/{version}", "wb") as f:
      pickle.dump(results, f)
  except:
    continue


/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/0.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/1.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/2.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/3.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/4.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/5.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/6.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/7.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/8.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/9.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/10.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/11.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/12.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/13.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/14.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/15.pth
/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/See

In [23]:
version = f'{name}Test.pkl'

In [ ]:
for i in range(0,25):
  try:
    path_to_model = f"/content/drive/MyDrive/TCC/Best_Models/{SPX}/{now}/Seeds/{i}.pth"
    print(path_to_model)

    best_tft.load_state_dict(torch.load(path_to_model, map_location=torch.device('cpu')))
    pred = calculate_sharpe(test, best_tft, max_encoder_length, max_prediction_length)
    results[i] = pred

    with open(f"/content/drive/MyDrive/TCC/Best_Models/Best_Results/New_Results/{SPX}/{version}", "wb") as f:
      pickle.dump(results, f)
  except:
    continue


/content/drive/MyDrive/TCC/Best_Models/BRL/BRL/Seeds/0.pth
